<a id=top></a>

# **目次**

<b>
    <details>
        <summary>
            <a href="#modules", style="font-size: xx-large">1. モジュールインポート</a>
            <ul>※サードパーティライブラリ>>>自作モジュール>>>（ここまで本ipynb外）>>>自作関数（本ipynb内）</ul>
        </summary>
    </details>
    <details>
        <summary>
            <a href="#data", style="font-size: xx-large">2. オリジナルデータインポート</a>
        </summary>
    </details>
    <details>
        <summary>
            <a href="#patentcount", style="font-size: xx-large">3. 特許数</a>
        </summary>
        <table></table>
    </details>
    <details>
        <summary>
            <a href="#calculateindicator", style="font-size: xx-large">4. 各指標</a>
        </summary>
    </details>
    <details>
        <summary>
            <a href="#output", style="font-size: xx-large">5. ファイルに出力</a>
        </summary>
    </details>
</b>


---


<a id=modules></a>

## **1. モジュールインポート**


In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../../src')
from ecomplexity import ecomplexity

# 小数点以下 桁数 6
pd.options.display.float_format = '{:.3f}'.format


In [2]:
import initial_condition


In [3]:
global data_dir, output_dir
data_dir = '../../data/interim/internal/filtered_after_agg/'
output_dir = '../../data/processed/internal/'
ex_dir = '../../data/processed/external/schmoch/'


In [4]:
# 初期条件
ar = initial_condition.AR
year_style = initial_condition.YEAR_STYLE

year_start = initial_condition.YEAR_START
year_end = initial_condition.YEAR_END
year_range = initial_condition.YEAR_RANGE

extract_population = initial_condition.EXTRACT_POPULATION
top_p_or_num = initial_condition.TOP_P_OR_NUM
top_p_or_num = ('p', 100)
region_corporation = initial_condition.REGION_CORPORATION
region_corporation = 'right_person_addr'
applicant_weight = initial_condition.APPLICANT_WEIGHT

classification = initial_condition.CLASSIFICATION
class_weight = initial_condition.CLASS_WEIGHT

input_condition = f'{ar}_{year_style}_{extract_population}_{top_p_or_num[0]}_{top_p_or_num[1]}_{region_corporation}_{applicant_weight}_{classification}_{class_weight}'
output_condition = f'{ar}_{year_style}_{extract_population}_{top_p_or_num[0]}_{top_p_or_num[1]}_{region_corporation}_{applicant_weight}_{classification}_{class_weight}'


In [5]:
input_condition

'app_nendo_all_p_100_right_person_addr_fraction_schmoch35_fraction'

In [6]:
def kh_ki(c_df, classification, n=19):
    kh1_ki1_df = pd.merge(c_df.copy(), 
                        c_df[c_df['mcp']==1].groupby([region_corporation])[['ubiquity']].sum().reset_index(drop=False).copy().rename(columns={'ubiquity':'kh_1'}), 
                        on=[region_corporation], how='left')
    kh1_ki1_df = pd.merge(kh1_ki1_df.copy(), 
                        c_df[c_df['mcp']==1].groupby([classification])[['diversity']].sum().reset_index(drop=False).copy().rename(columns={'diversity':'ki_1'}), 
                        on=[classification], how='left')
    kh1_ki1_df['kh_1'] = kh1_ki1_df['kh_1'] / kh1_ki1_df['diversity']
    kh1_ki1_df['ki_1'] = kh1_ki1_df['ki_1'] / kh1_ki1_df['ubiquity']
    kh_ki_df = kh1_ki1_df.copy()
    for i in range(n):
        kh_ki_df = pd.merge(kh_ki_df, 
                            kh_ki_df[kh_ki_df['mcp']==1].groupby([region_corporation])[[f'ki_{i+1}']].sum().reset_index(drop=False).copy()\
                                        .rename(columns={f'ki_{i+1}':f'kh_{i+2}'}), 
                            on=[region_corporation], how='left')
        kh_ki_df = pd.merge(kh_ki_df, 
                            kh_ki_df[kh_ki_df['mcp']==1].groupby([classification])[[f'kh_{i+1}']].sum().reset_index(drop=False).copy()\
                                        .rename(columns={f'kh_{i+1}':f'ki_{i+2}'}), 
                            on=[classification], how='left')
        kh_ki_df[f'kh_{i+2}'] = kh_ki_df[f'kh_{i+2}'] / kh_ki_df['diversity']
        kh_ki_df[f'ki_{i+2}'] = kh_ki_df[f'ki_{i+2}'] / kh_ki_df['ubiquity']
    return kh_ki_df


---


<a id=data></a>

## **2. オリジナルデータインポート**


In [7]:
schmoch_df = pd.read_csv(f'{ex_dir}35.csv', 
                         encoding='utf-8', 
                         sep=',', 
                         usecols=['Field_number', 'Field_en']
                         ).drop_duplicates()

In [8]:
reg_num_top_df = pd.read_csv(f'{data_dir}{input_condition}.csv', 
                             encoding='utf-8',
                             sep=',')
reg_num_top_df

,app_nendo_period,right_person_addr,schmoch35,reg_num
0,1981-2010,東京都,9,150440.498
1,1981-2010,東京都,2,107498.033
2,1981-2010,東京都,1,102029.614
3,1981-2010,東京都,6,97066.617
4,1981-2010,東京都,10,89819.198
...,...,...,...,...
6208,2001-2010,秋田県,7,0.500
6209,2001-2010,鹿児島県,5,0.500
6210,2001-2010,秋田県,22,0.500
6211,2001-2010,愛媛県,5,0.500


In [9]:
reg_num_top_df[region_corporation].nunique()

47

<a href=#top>先頭に戻る</a>

---


<a id=calculateindicator></a>

## **4. 各指標**


In [10]:
trade_cols = {'time':f'{ar}_{year_style}_period', 'loc':region_corporation, 'prod':classification, 'val':'reg_num'}
rename_col_dict = {'eci':'kci', 'pci':'tci'}
col_order_list = [f'{ar}_{year_style}_period', region_corporation, classification, 'reg_num', 'rca', 'mcp', 'diversity', 'ubiquity', 'kci', 'tci']


In [11]:
c_df = ecomplexity(reg_num_top_df,
                   cols_input = trade_cols, 
                   rca_mcp_threshold = 1)
# c_out_df = c_df.copy()
c_df = c_df[c_df['reg_num'] > 0]\
           .rename(columns=rename_col_dict)\
           [col_order_list]
c_df = pd.concat([kh_ki(c_df[c_df[f'{ar}_{year_style}_period'] == period], classification) for period in c_df[f'{ar}_{year_style}_period'].unique()], 
                 axis='index', 
                 ignore_index=True)

# for segment in c_df[f'{ar}_{year_style}_period'].unique():
#     display(c_df[c_df[f'{ar}_{year_style}_period'] == segment].head())
#     display(c_df[c_df[f'{ar}_{year_style}_period'] == segment].describe())
#     print(c_df[c_df[f'{ar}_{year_style}_period'] == segment].info())
#     print('\n')


1981-2010
1981-1990
1991-2000
2001-2010


In [12]:
c_df.sort_values(by=[f'{ar}_{year_style}_period', 'kci'], ascending=[True, False])

,app_nendo_period,right_person_addr,schmoch35,reg_num,rca,mcp,diversity,ubiquity,kci,tci,...,kh_16,ki_16,kh_17,ki_17,kh_18,ki_18,kh_19,ki_19,kh_20,ki_20
2608,1981-1990,神奈川県,1,7259.667,1.199,1,14,15,3.073,0.931,...,12.224,19.666,19.666,12.223,12.224,19.667,19.666,12.223,12.223,19.667
2609,1981-1990,神奈川県,2,7135.667,1.225,1,14,7,3.073,2.552,...,12.224,19.666,19.666,12.224,12.224,19.666,19.666,12.223,12.223,19.667
2610,1981-1990,神奈川県,3,3968.167,1.480,1,14,6,3.073,3.826,...,12.224,19.666,19.666,12.224,12.224,19.666,19.666,12.223,12.223,19.667
2611,1981-1990,神奈川県,4,1034.250,2.253,1,14,5,3.073,6.627,...,12.224,19.665,19.666,12.224,12.224,19.666,19.666,12.224,12.223,19.667
2612,1981-1990,神奈川県,5,2529.333,1.871,1,14,5,3.073,4.495,...,12.224,19.666,19.666,12.224,12.224,19.666,19.666,12.224,12.223,19.667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5728,2001-2010,神奈川県,31,2080.600,0.794,0,14,16,-3.998,0.591,...,13.589,22.905,22.905,13.589,13.589,22.905,22.905,13.589,13.589,22.905
5729,2001-2010,神奈川県,32,6553.250,1.464,1,14,10,-3.998,-1.725,...,13.589,22.905,22.905,13.589,13.589,22.905,22.905,13.589,13.589,22.905
5730,2001-2010,神奈川県,33,1898.833,0.422,0,14,6,-3.998,0.725,...,13.589,22.905,22.905,13.589,13.589,22.905,22.905,13.589,13.589,22.905
5731,2001-2010,神奈川県,34,923.500,0.607,0,14,16,-3.998,1.587,...,13.589,22.905,22.905,13.589,13.589,22.905,22.905,13.589,13.589,22.905


In [13]:
c_df[(c_df[classification]==22)&(c_df[f'{ar}_{year_style}_period']=='1981-1990')]

,app_nendo_period,right_person_addr,schmoch35,reg_num,rca,mcp,diversity,ubiquity,kci,tci,...,kh_16,ki_16,kh_17,ki_17,kh_18,ki_18,kh_19,ki_19,kh_20,ki_20
2201,1981-1990,岡山県,22,0.500,101.875,1,14,2,-0.839,0.455,...,12.223,19.666,19.667,12.224,12.223,19.667,19.667,12.223,12.223,19.667
2450,1981-1990,東京都,22,0.500,1.002,1,19,2,1.154,0.455,...,12.224,19.666,19.666,12.224,12.224,19.667,19.667,12.223,12.223,19.667


<a href="#top">先頭に戻る</a>

---


<a id=output></a>

## **5. ファイルに出力**

<a id=rightperson></a>

### **5.1. 特許権者**


In [14]:

right_person_df = pd.merge(c_df.groupby([f'{ar}_{year_style}_period', region_corporation])[['reg_num']].sum().reset_index(drop=False), 
                           c_df.groupby([f'{ar}_{year_style}_period', region_corporation])[[classification]].nunique().reset_index(drop=False), 
                           on=[f'{ar}_{year_style}_period', region_corporation], 
                           how='inner')
right_person_df = pd.merge(right_person_df, 
                           c_df[[f'{ar}_{year_style}_period', region_corporation, 'diversity', 'kci']\
                               +[f'kh_{i}' for i in range(1, 20+1)]]\
                               .drop_duplicates(keep='first'), 
                           on=[f'{ar}_{year_style}_period', region_corporation], 
                           how='inner')
# for period in right_person_df[f'{ar}_{year_style}_period'].unique():
#     right_person_df

# for period in right_person_df[f'{ar}_{year_style}_period'].unique():
#     for i in range(1, 20+1):
#         value = right_person_df[right_person_df[f'{ar}_{year_style}_period']==period]
#         right_person_df[right_person_df[f'{ar}_{year_style}_period']==period][f'kh_{i}'] = (value[f'kh_{i}'] - value[f'kh_{i}'].mean()) / value[f'kh_{i}'].std()
#     display(right_person_df[right_person_df[f'{ar}_{year_style}_period'] == period].head())
#     display(right_person_df[right_person_df[f'{ar}_{year_style}_period'] == period].describe())
#     print(right_person_df[right_person_df[f'{ar}_{year_style}_period'] == period].info())
#     print('\n')
# right_person_df['reg_num'] = right_person_df['reg_num'].astype(np.int64)

In [15]:
schmoch_df

,Field_number,Field_en
0,1,"Electrical machinery, apparatus, energy"
30,2,Audio-visual technology
48,3,Telecommunications
58,4,Digital communication
61,5,Basic communication processes
71,6,Computer technology
88,7,IT methods for management
89,8,Semiconductors
91,9,Optics
101,10,Measurement


In [16]:
sample = pd.merge(c_df, schmoch_df, 
         left_on=classification, right_on='Field_number', how='left')
sample[(sample[f'{ar}_{year_style}_period']==f'{year_start}-{year_end}')&(sample['Field_en']=='Digital communication')&\
    (sample['mcp']==1)]

,app_nendo_period,right_person_addr,schmoch35,reg_num,rca,mcp,diversity,ubiquity,kci,tci,...,kh_17,ki_17,kh_18,ki_18,kh_19,ki_19,kh_20,ki_20,Field_number,Field_en
37,1981-2010,京都府,4,1494.750,1.327,1,12,4,2.882,8.919,...,22.093,13.458,13.458,22.093,22.093,13.458,13.458,22.093,4,Digital communication
902,1981-2010,東京都,4,23829.950,1.250,1,19,4,1.885,8.919,...,22.093,13.458,13.458,22.093,22.093,13.458,13.458,22.093,4,Digital communication
1107,1981-2010,神奈川県,4,6570.617,2.119,1,13,4,3.587,8.919,...,22.093,13.458,13.458,22.093,22.093,13.458,13.458,22.093,4,Digital communication
1417,1981-2010,青森県,4,20.917,1.533,1,12,4,0.343,8.919,...,22.093,13.458,13.458,22.093,22.093,13.458,13.458,22.093,4,Digital communication


In [17]:
right_person_df[right_person_df[f'{ar}_{year_style}_period']==f'{year_start}-{year_end}'].sort_values('kci', ascending=False)

,app_nendo_period,right_person_addr,reg_num,schmoch35,diversity,kci,kh_1,kh_2,kh_3,kh_4,...,kh_11,kh_12,kh_13,kh_14,kh_15,kh_16,kh_17,kh_18,kh_19,kh_20
79,1981-2010,神奈川県,262459.708,35,13,3.587,11.308,13.408,19.463,13.460,...,22.084,13.458,22.091,13.458,22.092,13.458,22.093,13.458,22.093,13.458
48,1981-2010,京都府,95364.191,35,12,2.882,14.417,13.789,20.109,13.512,...,22.086,13.459,22.091,13.458,22.093,13.458,22.093,13.458,22.093,13.458
73,1981-2010,東京都,1613450.502,35,19,1.885,13.789,13.962,20.677,13.541,...,22.089,13.459,22.092,13.459,22.093,13.458,22.093,13.458,22.093,13.458
63,1981-2010,山梨県,6290.992,35,10,1.851,19.300,13.162,21.000,13.425,...,22.088,13.458,22.092,13.458,22.093,13.458,22.093,13.458,22.093,13.458
66,1981-2010,岩手県,1626.117,35,14,1.050,19.929,13.536,21.512,13.472,...,22.091,13.459,22.092,13.458,22.093,13.458,22.093,13.458,22.093,13.458
53,1981-2010,和歌山県,4381.917,35,12,0.931,20.500,13.471,21.597,13.476,...,22.091,13.459,22.093,13.458,22.093,13.458,22.093,13.458,22.093,13.458
62,1981-2010,山形県,4214.060,34,10,0.871,22.100,13.302,21.658,13.457,...,22.091,13.458,22.093,13.458,22.093,13.458,22.093,13.458,22.093,13.458
69,1981-2010,徳島県,4005.900,34,12,0.850,19.583,13.535,21.817,13.491,...,22.092,13.459,22.093,13.459,22.093,13.458,22.093,13.458,22.093,13.458
52,1981-2010,千葉県,19605.333,35,18,0.796,20.167,13.578,21.665,13.476,...,22.091,13.459,22.093,13.458,22.093,13.458,22.093,13.458,22.093,13.458
71,1981-2010,愛知県,237851.682,35,11,0.739,15.545,13.218,20.980,13.384,...,22.090,13.458,22.092,13.458,22.093,13.458,22.093,13.458,22.093,13.458


In [18]:
right_person_df.to_csv(f'{output_dir}firms/{output_condition}.csv', 
                       encoding='utf-8', 
                       sep=',', 
                       index=False)


In [19]:
right_person_df

,app_nendo_period,right_person_addr,reg_num,schmoch35,diversity,kci,kh_1,kh_2,kh_3,kh_4,...,kh_11,kh_12,kh_13,kh_14,kh_15,kh_16,kh_17,kh_18,kh_19,kh_20
0,1981-1990,三重県,1069.833,33,12,-0.563,21.833,12.123,20.169,12.213,...,19.671,12.223,19.668,12.223,19.667,12.223,19.667,12.223,19.667,12.223
1,1981-1990,京都府,20086.033,34,12,1.515,13.417,12.640,18.034,12.354,...,19.655,12.224,19.663,12.224,19.666,12.224,19.666,12.224,19.667,12.223
2,1981-1990,佐賀県,401.000,27,11,-1.030,21.636,12.156,20.418,12.222,...,19.674,12.223,19.669,12.223,19.667,12.223,19.667,12.223,19.667,12.223
3,1981-1990,兵庫県,22660.874,34,15,-0.240,19.333,12.023,19.988,12.156,...,19.669,12.223,19.667,12.223,19.667,12.223,19.667,12.223,19.667,12.223
4,1981-1990,北海道,1765.062,33,10,-1.240,24.500,12.024,20.762,12.190,...,19.676,12.223,19.670,12.223,19.668,12.223,19.667,12.223,19.667,12.223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,2001-2010,静岡県,19057.693,35,12,-0.782,20.000,13.290,22.150,13.538,...,22.902,13.589,22.904,13.589,22.905,13.589,22.905,13.589,22.905,13.589
184,2001-2010,香川県,1974.720,35,16,0.673,25.188,13.708,23.477,13.606,...,22.908,13.589,22.906,13.589,22.905,13.589,22.905,13.589,22.905,13.589
185,2001-2010,高知県,649.386,33,13,0.982,27.462,13.635,23.899,13.610,...,22.909,13.589,22.906,13.589,22.905,13.589,22.905,13.589,22.905,13.589
186,2001-2010,鳥取県,518.783,34,17,-0.099,23.765,13.593,22.967,13.590,...,22.905,13.589,22.905,13.589,22.905,13.589,22.905,13.589,22.905,13.589


<a href=#top>先頭に戻る</a>

---


<a id=ipc></a>

### **5.2. IPC**


In [20]:
schmoch_df = pd.read_csv(f'{ex_dir}35.csv', 
                         encoding='utf-8', 
                         sep=',', 
                         usecols=['Field_number', 'Field_en']
                         ).drop_duplicates()


In [21]:
# 各期間
classification_df = pd.merge(c_df.groupby([f'{ar}_{year_style}_period', classification])[['reg_num']].sum().reset_index(drop=False), 
                        c_df.groupby([f'{ar}_{year_style}_period', classification])[[region_corporation]].nunique().reset_index(drop=False), 
                        on=[f'{ar}_{year_style}_period', classification], 
                        how='inner')
classification_df = pd.merge(classification_df, 
                      c_df[[f'{ar}_{year_style}_period', classification, 'ubiquity', 'tci']\
                          +[f'ki_{i}' for i in range(1, 20+1)]]\
                          .drop_duplicates(keep='first'), 
                      on=[f'{ar}_{year_style}_period', classification], 
                      how='inner')
# classification_df['reg_num'] = classification_df['reg_num'].astype(np.int64)
classification_df = pd.merge(classification_df, 
                            schmoch_df.rename(columns={'Field_number':classification}), 
                            on=[classification], 
                            how='inner')\
                            .drop(columns=[classification])\
                            .rename(columns={'Field_en':classification})
display(classification_df)


,app_nendo_period,reg_num,right_person_addr,ubiquity,tci,ki_1,ki_2,ki_3,ki_4,ki_5,...,ki_12,ki_13,ki_14,ki_15,ki_16,ki_17,ki_18,ki_19,ki_20,schmoch35
0,1981-1990,56290.500,47,15,0.931,12.400,18.781,12.288,19.402,12.242,...,19.665,12.224,19.666,12.224,19.666,12.223,19.667,12.223,19.667,"Electrical machinery, apparatus, energy"
1,1981-2010,234113.000,47,17,2.140,12.882,20.416,13.399,21.659,13.447,...,22.092,13.458,22.093,13.458,22.093,13.458,22.093,13.458,22.093,"Electrical machinery, apparatus, energy"
2,1991-2000,69000.333,47,17,-1.398,12.353,20.426,12.633,20.534,12.668,...,20.566,12.674,20.566,12.674,20.566,12.674,20.566,12.674,20.566,"Electrical machinery, apparatus, energy"
3,2001-2010,108822.167,47,18,-1.485,12.944,21.528,13.488,22.525,13.570,...,22.903,13.589,22.905,13.589,22.905,13.589,22.905,13.589,22.905,"Electrical machinery, apparatus, energy"
4,1981-1990,54149.500,42,7,2.552,15.143,16.297,12.621,18.824,12.302,...,19.660,12.224,19.665,12.224,19.666,12.224,19.666,12.223,19.667,Audio-visual technology
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,2001-2010,21610.500,47,16,1.587,12.875,23.432,13.516,23.156,13.581,...,22.906,13.589,22.905,13.589,22.905,13.589,22.905,13.589,22.905,Other consumer goods
136,1981-1990,35240.500,47,28,-1.187,11.536,21.284,12.124,20.044,12.205,...,19.669,12.223,19.668,12.223,19.667,12.223,19.667,12.223,19.667,Civil engineering
137,1981-2010,153258.000,47,29,-1.787,13.138,23.891,13.441,22.461,13.459,...,22.094,13.458,22.093,13.458,22.093,13.458,22.093,13.458,22.093,Civil engineering
138,1991-2000,63870.833,47,30,0.525,12.133,22.343,12.557,20.895,12.653,...,20.567,12.674,20.566,12.674,20.566,12.674,20.566,12.674,20.566,Civil engineering


In [22]:
classification_df.to_csv(f'{output_dir}technology/{output_condition}.csv', 
                        encoding='utf-8', 
                        sep=',', 
                        index=False)


<a href=#top>先頭に戻る</a>

---


<a id=network></a>

## **5.3. 二部グラフ用**


In [31]:
eneos_df = c_df[(c_df[f'{ar}_{year_style}_period']==f'{year_start}-{year_end}')&(c_df[region_corporation].str.contains('ＥＮＥＯＳ'))\
                &(c_df['mcp']==1)].copy()#[[region_corporation, 'reg_num', 'schmoch35']].copy()
eneos_df = pd.merge(eneos_df, 
                    schmoch_df.rename(columns={'Field_number':'schmoch35'})\
                              .drop_duplicates(keep='first'), 
                    on=['schmoch35'], 
                    how='inner')
eneos_df[['ubiquity', 'Field_en', 'ki_1']]

,ubiquity,Field_en,ki_1


In [24]:
c_df

,app_nendo_period,right_person_addr,schmoch35,reg_num,rca,mcp,diversity,ubiquity,kci,tci,...,kh_16,ki_16,kh_17,ki_17,kh_18,ki_18,kh_19,ki_19,kh_20,ki_20
0,1981-2010,三重県,1,4604.083,6.138,1,8,17,-0.591,2.140,...,13.458,22.093,22.093,13.458,13.458,22.093,22.093,13.458,13.458,22.093
1,1981-2010,三重県,2,222.417,0.360,0,8,9,-0.591,3.020,...,13.458,22.093,22.093,13.458,13.458,22.093,22.093,13.458,13.458,22.093
2,1981-2010,三重県,3,33.000,0.100,0,8,6,-0.591,6.395,...,13.458,22.093,22.093,13.458,13.458,22.093,22.093,13.458,13.458,22.093
3,1981-2010,三重県,4,41.500,0.344,0,8,4,-0.591,8.919,...,13.458,22.093,22.093,13.458,13.458,22.093,22.093,13.458,13.458,22.093
4,1981-2010,三重県,5,15.500,0.134,0,8,7,-0.591,6.208,...,13.458,22.093,22.093,13.458,13.458,22.093,22.093,13.458,13.458,22.093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6208,2001-2010,鹿児島県,31,4.792,0.252,0,16,16,0.534,0.591,...,13.589,22.905,22.905,13.589,13.589,22.905,22.905,13.589,13.589,22.905
6209,2001-2010,鹿児島県,32,5.000,0.154,0,16,10,0.534,-1.725,...,13.589,22.905,22.905,13.589,13.589,22.905,22.905,13.589,13.589,22.905
6210,2001-2010,鹿児島県,33,14.000,0.430,0,16,6,0.534,0.725,...,13.589,22.905,22.905,13.589,13.589,22.905,22.905,13.589,13.589,22.905
6211,2001-2010,鹿児島県,34,22.500,2.041,1,16,16,0.534,1.587,...,13.589,22.905,22.905,13.589,13.589,22.905,22.905,13.589,13.589,22.905


In [25]:
graph_df = pd.merge(c_df, schmoch_df, 
                    left_on=classification, right_on='Field_number', how='left')\
                    .drop(columns=['Field_number', classification])\
                    .rename(columns={'Field_en':classification})
# graph_df = graph_df[graph_df['mcp']==1][[f'{ar}_{year_style}', region_corporation, 'ipc_class', 'mcp']]
graph_df

,app_nendo_period,right_person_addr,reg_num,rca,mcp,diversity,ubiquity,kci,tci,kh_1,...,ki_16,kh_17,ki_17,kh_18,ki_18,kh_19,ki_19,kh_20,ki_20,schmoch35
0,1981-2010,三重県,4604.083,6.138,1,8,17,-0.591,2.140,26.000,...,22.093,22.093,13.458,13.458,22.093,22.093,13.458,13.458,22.093,"Electrical machinery, apparatus, energy"
1,1981-2010,三重県,222.417,0.360,0,8,9,-0.591,3.020,26.000,...,22.093,22.093,13.458,13.458,22.093,22.093,13.458,13.458,22.093,Audio-visual technology
2,1981-2010,三重県,33.000,0.100,0,8,6,-0.591,6.395,26.000,...,22.093,22.093,13.458,13.458,22.093,22.093,13.458,13.458,22.093,Telecommunications
3,1981-2010,三重県,41.500,0.344,0,8,4,-0.591,8.919,26.000,...,22.093,22.093,13.458,13.458,22.093,22.093,13.458,13.458,22.093,Digital communication
4,1981-2010,三重県,15.500,0.134,0,8,7,-0.591,6.208,26.000,...,22.093,22.093,13.458,13.458,22.093,22.093,13.458,13.458,22.093,Basic communication processes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6208,2001-2010,鹿児島県,4.792,0.252,0,16,16,0.534,0.591,24.688,...,22.905,22.905,13.589,13.589,22.905,22.905,13.589,13.589,22.905,Mechanical elements
6209,2001-2010,鹿児島県,5.000,0.154,0,16,10,0.534,-1.725,24.688,...,22.905,22.905,13.589,13.589,22.905,22.905,13.589,13.589,22.905,Transport
6210,2001-2010,鹿児島県,14.000,0.430,0,16,6,0.534,0.725,24.688,...,22.905,22.905,13.589,13.589,22.905,22.905,13.589,13.589,22.905,"Furniture, games"
6211,2001-2010,鹿児島県,22.500,2.041,1,16,16,0.534,1.587,24.688,...,22.905,22.905,13.589,13.589,22.905,22.905,13.589,13.589,22.905,Other consumer goods


In [27]:
graph_df.to_csv(f'{output_dir}graph/{output_condition}.csv', 
                encoding='utf-8', 
                sep=',', 
                index=False)
# graph_df
